In [ ]:
import torch 
import numpy as np
import config
from Plotting.myplt import plot_lines,plot_lines_with_colors
import tslearn
from tslearn import clustering
import matplotlib
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import numba
from numba import jit
from sklearn.cluster import DBSCAN, OPTICS
import networkx as nx
import graph_layout

@jit(nopython=True)
def code_distance(x1:np.array,x2:np.array):
    d = 0
    for i in range(len(x1)):
        d += int(x1[i]!=x2[i])
    return d/len(x1)

def get_distance_matrix(X, metric):
    N_ = len(X)
    d_m = np.zeros(shape=(N_,N_))
    for i in range(N_-1):
        print('\r{}/{}'.format(i,N_-2),end='')
        for j in range(i+1,N_):
            d_ = metric(X[i],X[j])
            d_m[i][j] = d_
            d_m[j][i] = d_
    return d_m
def get_all_distances(X, metric):
    N_ = len(X)
    distances_ = np.zeros(shape=(int(N_*(N_-1)/2),))
    k_ = 0
    for i in range(N_-1):
        print('\r{}/{}'.format(i,N_-2),end='')
        for j in range(i+1,N_):
            distances_[k_] = metric(X[i],X[j])
            k_ += 1
    return distances_

def plot_float_distribution(data,fig_size=(4,3),title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(fig_size[0],fig_size[1])
    x = []
    for i in range(len(data)):
        if np.isnan(data[i]):
            continue
        else:
            x.append(data[i])
    u_vs = np.unique(x)

    if len(x) == 0:
        ax.set_title(title +' is empty data')
    elif len(u_vs)==1:
        ax.set_title(title + ' all data is repeated with value: {}'.format(u_vs[0]))
    else:
        x = np.asarray(x)
        q25, q75 = np.percentile(x, [25, 75])
        bins = 0
        if q25==q75:
            bins = np.minimum(100,len(u_vs))
        else:
            bin_width = 2 * (q75 - q25) * len(x) ** (-1 / 3)
            bins = np.minimum(100, round((np.max(x) - np.min(x)) / bin_width))
        nan_rate = np.sum(np.isnan(data))/len(data)
        ax.set_title(title+'. n of unique values {}'.format(len(u_vs)))
        ax.set_xlabel('nan rate {}'.format(nan_rate))
        density,bins = np.histogram(x,bins=bins,density=True)
        unity_density = density / density.sum()
        widths = bins[:-1] - bins[1:]
        ax.bar(bins[1:], unity_density,width=widths)

    return fig,ax

def get_best_K(data):
    distances_to_centroids = []
    sil_score = []
    number_of_clusters = []
    for K in tqdm(range(3,8)):
        number_of_clusters.append(K)
        kMeansModel = clustering.TimeSeriesKMeans(n_clusters=K,n_jobs=8,max_iter=10,metric='euclidean')
        kMeansModel.fit(data)
        distances_to_centroids.append(kMeansModel.inertia_)
        sil_score.append(clustering.silhouette_score(data,kMeansModel.labels_,n_jobs=8,metric='euclidean'))
    distances_to_centroids = np.array(distances_to_centroids)
    sil_score = np.array(sil_score)

    # n1 = 10
    # segments_d = np.linspace(start= np.min(distances_to_centroids),stop=np.max(distances_to_centroids), num=n1+1)
    # D_q = np.zeros(shape=(len(distances_to_centroids,)))
    # for i in range(len(distances_to_centroids)):
    #     for j in range(len(segments_d)-1):
    #         if j == len(segments_d)-1:
    #             if distances_to_centroids[i] >= segments_d[j] and distances_to_centroids[i] <= segments_d[j+1]:
    #                 D_q[i] = n1 - j 
    #                 break 
    #         else:
    #             if distances_to_centroids[i] >= segments_d[j] and distances_to_centroids[i] < segments_d[j+1]:
    #                 D_q[i] = n1 - j 
    #                 break
    D_q = distances_to_centroids
    D_q = D_q / np.max(D_q)
    D_q = 1.0-D_q
    # dDdk = np.zeros(shape=(len(sil_score),))
    # for i in range(len(sil_score)):
    #     if i==0:
    #         dDdk[i] = (distances_to_centroids[i+1]-distances_to_centroids[i])/(number_of_clusters[i+1]-number_of_clusters[i])
    #     elif i== len(sil_score)-1:
    #         dDdk[i] = (distances_to_centroids[i]-distances_to_centroids[i-1])/(number_of_clusters[i]-number_of_clusters[i-1])
    #     else:
    #         h1 = number_of_clusters[i+1]-number_of_clusters[i]
    #         h2 = number_of_clusters[i-1]-number_of_clusters[i]
    #         y_i_plus_1 = distances_to_centroids[i+1] 
    #         y_i = distances_to_centroids[i]
    #         y_i_minus_1=  distances_to_centroids[i-1]
    #         dDdk[i] = (h2*y_i_plus_1-h1*y_i_minus_1+ (h1-h2)*y_i)/(2*h1*h2)
    # dDdk = np.abs(dDdk)
    # dDdk = np.power(dDdk,-1)
    # dDdk = dDdk/np.max(dDdk)

    # fig,ax = plt.subplots(nrows=4,ncols=1)
    # fig.set_size_inches(16,16)
    # ax[0].plot(number_of_clusters, D_q)
    # ax[0].set_title(r'$D_{quality} \: scaled$')
    # ax[0].set_xlabel('K')
    # ax[0].set_xticks(number_of_clusters)

    # ax[1].plot(number_of_clusters,distances_to_centroids)
    # ax[1].set_title(r'$D$')
    # ax[1].set_xlabel('K')
    # ax[1].set_xticks(number_of_clusters)

    # ax[2].plot(number_of_clusters,sil_score)
    # ax[2].set_title('sil_score')
    # ax[2].set_xlabel('K')
    # ax[2].set_xticks(number_of_clusters)

    # k_vec = np.array(number_of_clusters)
    # K_loss = 1.0 - k_vec/np.max(k_vec)

    # Q = np.zeros(shape=(len(sil_score,)))
    # for i in range(len(sil_score)):
    #     Q[i] = np.minimum(D_q[i],sil_score[i])*K_loss[i]   
    # ax[3].plot(number_of_clusters,Q)
    # ax[3].set_title(r'$Q$')
    # ax[3].set_xlabel('K')
    # ax[3].set_xticks(number_of_clusters)

    k_vec = np.array(number_of_clusters)
    K_loss = 1.0 - k_vec/np.max(k_vec)

    Q = np.zeros(shape=(len(sil_score,)))
    for i in range(len(sil_score)):
        Q[i] = np.minimum(D_q[i],sil_score[i]) 

    best_k = number_of_clusters[np.argmax(Q)]
    return best_k

In [ ]:
dataset = torch.load(config.dataset_for_time_clustering_path)
Theta = dataset['params']
Y =  dataset['solutions']
names = dataset['names']
index_by_name = dataset['index_by_name']
name_by_index = dataset['name_by_index']
print('Theta {}\tY {}'.format(Theta.shape, Y.shape))
print(names)

# multivariate time series clustering

## scaling

In [ ]:
# for i in range(len(Y)):
#     for j in range(len(names)):
#         a_ = np.min(Y[i,:,j])
#         b_ = np.max(Y[i,:,j])
#         Y_tilde = (Y[i,:,j]-a_)/(b_-a_)
#         Y[i,:,j] = Y_tilde

## time series to series of differences

In [ ]:
# Y_ = np.zeros(shape=(Y.shape[0],Y.shape[1]-1,Y.shape[2]))
# for i in range(len(Y)):
#     for j in range(len(names)):
#         Y_tilde = np.diff(Y[i,:,j])
#         Y_[i,:,j] = Y_tilde
# Y = Y_
# print(Y.shape)

## kmeans

In [ ]:
object_labels = np.zeros(shape=(len(names),len(Y)), dtype=np.intc)
best_k = np.zeros(shape=(len(names),))
for i in range(len(names)):
    print('{}/{}'.format(i+1, len(names)))
    name_i = names[i]
    data = Y[:,:,index_by_name[name_i]]
    # K = get_best_K(data)
    K = 7
    kMeansModel = clustering.TimeSeriesKMeans(n_clusters=K,n_jobs=8,max_iter=100,metric='euclidean')
    kMeansModel.fit(data)
    labels_ = kMeansModel.labels_
    unique_labels_ = np.unique(labels_)
    object_labels[i] = labels_
    best_k[i] = K
    print('num of clusters {}'.format(len(unique_labels_)))
    cmap = matplotlib.colors.ListedColormap(sns.color_palette("bright", len(unique_labels_)).as_hex())
    colors_ = [cmap(el) for el in labels_]
    fig,ax = plot_lines_with_colors(data,title='$'+name_i+' \: $',colors=['k' for j_ in range(len(data))], alpha = 0.1)
    for cluster_index in range(K):
        # print(kMeansModel.cluster_centers_[cluster_index])
        # print(kMeansModel.cluster_centers_[cluster_index,:,0].flatten())
        ax.plot(kMeansModel.cluster_centers_[cluster_index,:,0].flatten(),color=cmap(cluster_index),alpha=1.0,label= str(cluster_index))
    ax.legend()
    ax.set_ylabel('ммоль/литр')
    ax.set_xlabel('мин')
    ax.grid(which = "both")
    ax.minorticks_on()
    ax.tick_params(which = "minor", bottom = False, left = False)
    fig.savefig(fname='./to_pr/to_pr_{}.png'.format(i),dpi = 200)
    # plot centroids 
    plt.show()
    # raise SystemExit
print(best_k)


In [ ]:
raise SystemExit

In [ ]:
agg_labels = np.zeros(shape=(len(Y),len(names)),dtype=np.intc)
for i in range(len(Y)):
    object_code = np.zeros(shape=(len(names),),dtype=np.intc)
    for j in range(len(names)):
        object_code[j] = object_labels[j,i]
    agg_labels[i] = object_code
print('number of clusters {}'.format(len(np.unique(agg_labels,axis=0))))
torch.save(agg_labels, r'/home/user/gr_lab/data/labels_after_clustering.txt')

# cluster labels with OPTICS

In [ ]:
X = torch.load(r'/home/user/gr_lab/data/labels_after_clustering.txt')
distances = get_all_distances(X,metric=code_distance)
plot_float_distribution(distances)
distance_matrix = get_distance_matrix(X,metric=code_distance)

In [ ]:
min_s = []
number_of_clusters = []
rate_of_unknown_objects = []
for MIN_S in tqdm(range(2,20)):
    clustering_alg = OPTICS(min_samples=MIN_S,metric=code_distance)
    clustering_alg.fit(X)
    clusters_labels = np.unique(clustering_alg.labels_)
    number_of_clusters.append(len(clusters_labels))
    min_s.append(MIN_S)
    rate_of_unknown_objects.append(np.sum((clustering_alg.labels_ == -1))/len(clustering_alg.labels_))
    # print('number of clusters {}'.format(len(clusters_labels)))

In [ ]:
fig,ax = plt.subplots(nrows=3,ncols=1)
fig.set_size_inches(16,30)
ax[0].plot(min_s, number_of_clusters)
ax[0].set_yticks(number_of_clusters)
ax[0].set_xticks(min_s)
ax[0].set_title('number_of_clusters(min samples)')

ax[1].plot(min_s, rate_of_unknown_objects)
ax[1].set_yticks(rate_of_unknown_objects)
ax[1].set_xticks(min_s)
ax[1].set_title('rate_of_unknown_objects(min samples)')

N_q = 1.0 - number_of_clusters/np.max(number_of_clusters)
R_q = 1.0 - rate_of_unknown_objects/np.max(rate_of_unknown_objects)
Q_optics = np.minimum(N_q, R_q)
ax[2].plot(min_s, Q_optics)
ax[2].set_yticks(Q_optics)
ax[2].set_xticks(min_s)
ax[2].set_title('Q(min samples)')
# Q_optics = np.zeros(shape=(len(min_s),))
# for i in range(len(min_s)):
    # Q_optics[i] = np.minimum(N_q[i],R_q)

In [ ]:
clustering_alg_ = OPTICS(min_samples=14,metric=code_distance)
clustering_alg_.fit(X)
clusters_labels_ = clustering_alg_.labels_
unique_labels_ = np.unique(clusters_labels_)
N_ = len(np.unique(clusters_labels_))

In [ ]:
print(N_)
cmap = matplotlib.colors.ListedColormap(sns.color_palette("bright", N_).as_hex())
colors_ = []
for i in range(len(clusters_labels_)):
    label_ = clusters_labels_[i]
    if label_ == -1: 
        colors_.append((0.0,0.0,0.0,1.0))
    else:
        colors_.append(cmap(label_))
unique_labels_colors_ = []
for i in range(len(unique_labels_)):
    color_ = 0
    if unique_labels_[i] == -1:
        color_ = (0.0,0.0,0.0,1.0)
    else:
        color_ = cmap(unique_labels_[i])
    unique_labels_colors_.append(color_)

In [ ]:
print((np.sum(clusters_labels_==-1)))